In [1]:
import kuti
from kuti import model_helper as mh
from kuti import applications as apps
from kuti import tensor_ops as ops
from kuti import generic as gen
from kuti import image_utils as img
import pandas as pd, numpy as np, os
from sklearn.metrics import r2_score


Loaded Kuti


In [2]:
root_path = '/media/workstation/0832621B32620DCE/Ian/'
dataset = root_path + 'mtaiq/PARA_MTAIQ_All_User_official_dataset.csv'
images_path = '/media/workstation/0832621B32620DCE/PARA_Dataset/PARA/PARA_resized/'
ids = pd.read_csv(dataset)

print('ID for PARA Dataset')
ids

ID for PARA Dataset


,sessionId,imageName,aestheticScore,qualityScore,sessionId_imageName,set
0,session328,iaa_pub22893_.jpg,1.980000,2.084000,session328_iaa_pub22893_.jpg,training
1,session232,iaa_pub16180_.jpg,3.720000,3.840000,session232_iaa_pub16180_.jpg,training
2,session125,iaa_pub8721_.jpg,2.900000,3.052000,session125_iaa_pub8721_.jpg,training
3,session139,iaa_pub9725_.jpg,3.120000,3.356000,session139_iaa_pub9725_.jpg,training
4,session79,iaa_pub5474_.jpg,3.840000,3.932000,session79_iaa_pub5474_.jpg,training
...,...,...,...,...,...,...
31215,session7,iaa_pub484_.jpg,3.540000,3.664000,session7_iaa_pub484_.jpg,validation
31216,session68,iaa_pub4700_.jpg,2.640000,2.984000,session68_iaa_pub4700_.jpg,validation
31217,session20,iaa_pub1377_.jpg,3.340000,3.428000,session20_iaa_pub1377_.jpg,validation
31218,session64,iaa_pub4477_.jpg,3.464286,3.607143,session64_iaa_pub4477_.jpg,validation


In [3]:
# define the MLSP wide model (updated from kuti.applications)
from kuti.applications import InceptionResNetV2, Lambda, tf, Concatenate, Model,\
Input, GlobalAveragePooling2D

def model_inceptionresnet_pooled(input_shape=(None, None, 3), indexes=list(range(43)),
                                 pool_size=(5, 5), name='', return_sizes=False):
    """
    Returns the wide MLSP features, spatially pooled, from InceptionResNetV2.
    * input_shape: shape of the input images
    * indexes: indices of the modules to use
    * pool_size: spatial extend of the MLSP features
    * name: name of the model
    * return_sizes: return the sizes of each layer: (model, pool_sizes)
    :return: model or (model, pool_sizes)
    """

    print('Loading InceptionResNetV2 multi-pooled with input_shape:', input_shape)
    model_base = InceptionResNetV2(weights     = 'imagenet',
                                   include_top = False,
                                   input_shape = input_shape)
    print('Creating multi-pooled model')

    ImageResizer = Lambda(lambda x: tf.image.resize(x, pool_size, method='area'),
                          name='feature_resizer')

    feature_layers = [l for l in model_base.layers if 'mixed' in l.name]
    feature_layers = [feature_layers[i] for i in indexes]
    pools = [ImageResizer(l.output) for l in feature_layers]
    conc_pools = Concatenate(name='conc_pools', axis=3)(pools)

    model = Model(inputs  = model_base.input, outputs = conc_pools)
    if name: model.name = name

    if return_sizes:
        pool_sizes = [[np.int32(x) for x in f.get_shape()[1:]] for f in pools]
        return model, pool_sizes
    else:
        return model

In [4]:
# load base model
model_name = 'mlsp_wide_orig'
input_shape = (None, None, 3)
model_base = model_inceptionresnet_pooled(input_shape)
pre = apps.process_input[apps.InceptionResNetV2]
print("InceptionResNetV2 loaded")

# MODEL DEF
input_feats = Input(shape=(5,5,16928), dtype='float32')
x = apps.inception_block(input_feats, size=1024)
x = GlobalAveragePooling2D(name='final_GAP')(x)

# pred = apps.fc_layers(x, name       = 'head',
#                       fc_sizes      = [2048, 1024, 256,  1],
#                       dropout_rates = [0.25, 0.25, 0.5, 0],
#                       batch_norm    = 2)

fc1_size = 2048

bn = 2
fc_sizes = [fc1_size, fc1_size/2, fc1_size/8,  1]
dropout_rates = [0.25, 0.25, 0.5, 0]

pred_aesthetic = apps.fc_layers(x, name       = 'head_aesthetic',
                                fc_sizes      = fc_sizes,
                                dropout_rates = dropout_rates,
                                batch_norm    = bn)

pred_quality = apps.fc_layers(x, name       = 'head_quality',
                              fc_sizes      = fc_sizes,
                              dropout_rates = dropout_rates,
                              batch_norm    = bn)                          

model = Model(inputs=input_feats, outputs=[pred_aesthetic, pred_quality]) #

gen_params = dict(batch_size    = 1,
                  data_path     = images_path,                  
                  process_fn    = pre,
                  input_shape   = input_shape,
                  inputs        = ['sessionId_imageName'],
                  outputs       = [['aestheticScore'], ['qualityScore']], 
                  fixed_batches = False)

helper = mh.ModelHelper(model, model_name, ids, 
                        gen_params = gen_params)

# print(helper.params.models_roots)
# load head model
helper.load_model(model_name = '/mnt/956cc712-57b2-4a24-b574-3a113e957774/Ian_temp/models_all_user/irnv2_mlsp_wide_orig/bn:2 bsz:128 do:[0.25,0.25,0.5,0] ds:[PARA_MTAIQ_All_User_official_da fc1:[2048] i:1[5,5,16928] im:[orig] l:[MSE,MSE] mon:[val_head_aesthetic_out_plcc_tf] o:2[1]')

# join base and head models
helper.model = Model(inputs  = model_base.input, 
                     outputs = model(model_base.output))

Loading InceptionResNetV2 multi-pooled with input_shape: (None, None, 3)


2023-02-15 22:04:14.669241: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-15 22:04:14.691070: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-15 22:04:14.691251: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-15 22:04:14.691748: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

Creating multi-pooled model
InceptionResNetV2 loaded
Model weights loaded: /mnt/956cc712-57b2-4a24-b574-3a113e957774/Ian_temp/models_all_user/irnv2_mlsp_wide_orig/bn:2 bsz:128 do:[0.25,0.25,0.5,0] ds:[PARA_MTAIQ_All_User_official_da fc1:[2048] i:1[5,5,16928] im:[orig] l:[MSE,MSE] mon:[val_head_aesthetic_out_plcc_tf] o:2[1]_best_weights.h5


### Predict multiple images

In [56]:
# Change so that user only needs to input image name to predict the score. 

best_image_list =  ['session277_iaa_pub19358_.jpg', 'session282_iaa_pub19732_.jpg',
                    'session194_iaa_pub13534_.jpg','session134_iaa_pub9338_.jpg',
                    'session119_iaa_pub8297_.jpg','session81_iaa_pub5613_.jpg']

worse_image_list = ['session57_iaa_pub3950_.jpg', 'session50_iaa_pub3492_.jpg',
                    'session50_iaa_pub3492_.jpg', 'session2_iaa_pub124_.jpg',
                    'session304_iaa_pub21227_.jpg', 'session127_iaa_pub8867_.jpg']

ids_copy = ids.copy(deep=True)
predict_aesthetic_best = []
predict_quality_best = []
predict_aesthetic_worse = []
predict_quality_worse = []

# predict best images
for i in range(len(best_image_list)):
    image_path = images_path + best_image_list[i]
    I = pre( img.read_image(image_path) )
    I = np.expand_dims(I, 0)
    I_score_best = helper.model.predict(I)

    predict_aesthetic_best.append(I_score_best[0][0])
    predict_quality_best.append(I_score_best[1][0])

for i in range(len(worse_image_list)):
    image_path = images_path + worse_image_list[i]
    I = pre( img.read_image(image_path) )
    I = np.expand_dims(I, 0)
    I_score_worse = helper.model.predict(I)

    predict_aesthetic_worse.append(I_score_worse[0][0])
    predict_quality_worse.append(I_score_worse[1][0])

In [57]:
for i in range(len(best_image_list)):
    true_aesthetic = ids_copy.loc[ids_copy.sessionId_imageName==best_image_list[i], 'aestheticScore'].values[0]
    true_quality = ids_copy.loc[ids_copy.sessionId_imageName==best_image_list[i], 'qualityScore'].values[0]
    print(f'Image {best_image_list[i]}: \nTrue Aesthetic: {true_aesthetic:.3f}, True Quality: {true_quality:.3f} \nPredicted Aesthetic: {predict_aesthetic_best[i].item():.3f}, Predicted Quality: {predict_quality_best[i].item():.3f} \n')


Image session277_iaa_pub19358_.jpg: 
True Aesthetic: 1.520, True Quality: 1.608 
Predicted Aesthetic: 1.514, Predicted Quality: 1.606 

Image session282_iaa_pub19732_.jpg: 
True Aesthetic: 3.180, True Quality: 3.320 
Predicted Aesthetic: 3.180, Predicted Quality: 3.401 

Image session194_iaa_pub13534_.jpg: 
True Aesthetic: 3.360, True Quality: 3.592 
Predicted Aesthetic: 3.301, Predicted Quality: 3.593 

Image session134_iaa_pub9338_.jpg: 
True Aesthetic: 3.180, True Quality: 3.400 
Predicted Aesthetic: 3.179, Predicted Quality: 3.399 

Image session119_iaa_pub8297_.jpg: 
True Aesthetic: 2.125, True Quality: 2.229 
Predicted Aesthetic: 2.127, Predicted Quality: 2.218 

Image session81_iaa_pub5613_.jpg: 
True Aesthetic: 3.160, True Quality: 3.284 
Predicted Aesthetic: 3.158, Predicted Quality: 3.370 



In [54]:
import cv2

for i in range(len(worse_image_list)):
    true_aesthetic = ids_copy.loc[ids_copy.sessionId_imageName==worse_image_list[i], 'aestheticScore'].values[0]
    true_quality = ids_copy.loc[ids_copy.sessionId_imageName==worse_image_list[i], 'qualityScore'].values[0]
    print(f'Image {worse_image_list[i]}: \nTrue Aesthetic: {true_aesthetic:.3f}, True Quality: {true_quality:.3f} \nPredicted Aesthetic: {predict_aesthetic_worse[i].item():.3f}, Predicted Quality: {predict_quality_worse[i].item():.3f} \n')


Image session57_iaa_pub3950_.jpg: 
True Aesthetic: 3.339, True Quality: 3.357 
Predicted Aesthetic: 2.233, Predicted Quality: 2.334 

Image session50_iaa_pub3492_.jpg: 
True Aesthetic: 3.120, True Quality: 3.296 
Predicted Aesthetic: 2.146, Predicted Quality: 2.072 

Image session50_iaa_pub3492_.jpg: 
True Aesthetic: 3.120, True Quality: 3.296 
Predicted Aesthetic: 2.146, Predicted Quality: 2.072 

Image session2_iaa_pub124_.jpg: 
True Aesthetic: 1.640, True Quality: 1.744 
Predicted Aesthetic: 2.721, Predicted Quality: 2.960 

Image session304_iaa_pub21227_.jpg: 
True Aesthetic: 2.300, True Quality: 2.212 
Predicted Aesthetic: 3.320, Predicted Quality: 3.530 

Image session127_iaa_pub8867_.jpg: 
True Aesthetic: 3.440, True Quality: 3.564 
Predicted Aesthetic: 2.600, Predicted Quality: 2.772 

